In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np

In [2]:
year = 2020

url = f"https://www.basketball-reference.com/leagues/NBA_{year}_standings.html"

driver = webdriver.Chrome()

driver.get(url)

driver.implicitly_wait(100)

page = driver.page_source

soup = BeautifulSoup(page, 'html')

In [3]:
table = soup.findAll('table', id="expanded_standings", limit=1)[0]


In [4]:
headers = table.find_all("th")
header_list = []

for header in headers:
    header_list.append(header.text)

table_headers_start = header_list.index('Rk')
table_headers_end = header_list.index('Aug') + 1

table_headers = header_list[table_headers_start:table_headers_end]

table_headers.remove('Jul')
table_headers.remove('Aug')
table_headers.append('Apr')
print(table_headers)

['Rk', 'Team', 'Overall', 'Home', 'Road', 'E', 'W', 'A', 'C', 'SE', 'NW', 'P', 'SW', 'Pre', 'Post', '≤3', '≥10', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr']


In [5]:
df = pd.DataFrame(columns=table_headers)

In [6]:
column_data = table.findAll("tr")

In [7]:
rank_start = header_list.index('1')

ranks = header_list[rank_start:]

In [8]:
def add_records(x, y):
    if x == '':
        record1 = np.array([0,0])
    else:
        record1_str = x.split("-")
        record1 = np.array([int(n) for n in record1_str])
    
    if y == '':
        record2 = np.array([0,0])
    else:
        record2_str = y.split("-")
        record2 = np.array([int(n) for n in record2_str])

    added_records = record1 + record2

    return f"{added_records[0]}-{added_records[1]}"



In [9]:
i = 0

for column in column_data:
    cells = column.find_all("td")

    individual_column_data = [cell.text.strip() for cell in cells]

    if individual_column_data == []:
        continue

    individual_column_data.insert(0, ranks[i])
    record1 = individual_column_data[-2]
    record2 = individual_column_data[-1]

    apr_record = add_records(record1, record2)
    individual_column_data.pop(-1)
    individual_column_data.pop(-1)

    individual_column_data.append(apr_record)
    print(individual_column_data)
    length = len(df)

    df.loc[length] = individual_column_data
    i += 1

1-0
2-5
['1', 'Milwaukee Bucks', '56-17', '30-5', '26-12', '37-7', '19-10', '10-4', '13-1', '14-2', '7-3', '7-2', '5-5', '46-8', '10-9', '3-2', '38-8', '2-2', '15-1', '13-2', '11-2', '10-1', '2-4', '3-5']

7-1
['2', 'Toronto Raptors', '53-19', '26-10', '27-9', '34-11', '19-8', '9-5', '13-3', '12-3', '7-3', '7-2', '5-3', '40-15', '13-4', '6-4', '23-10', '4-1', '10-3', '9-7', '12-3', '7-3', '4-1', '7-1']
1-0
2-5
['3', 'Los Angeles Lakers', '52-19', '25-10', '27-9', '16-9', '36-10', '5-5', '4-3', '7-1', '12-3', '10-3', '14-4', '41-12', '11-7', '7-3', '25-11', '3-1', '14-1', '9-5', '10-4', '9-2', '4-1', '3-5']
0-1
5-2
['4', 'Los Angeles Clippers', '49-23', '27-9', '22-14', '17-7', '32-16', '5-3', '4-3', '8-1', '11-5', '8-6', '13-5', '37-18', '12-5', '8-3', '32-8', '4-2', '10-4', '10-5', '9-4', '7-4', '4-1', '5-3']
0-1
5-2
['5', 'Boston Celtics', '48-24', '26-10', '22-14', '30-13', '18-11', '9-6', '9-4', '12-3', '6-3', '6-4', '6-4', '38-16', '10-8', '6-5', '29-7', '3-1', '10-4', '10-3', '9-

In [10]:
df

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,Milwaukee Bucks,56-17,30-5,26-12,37-7,19-10,10-4,13-1,14-2,...,10-9,3-2,38-8,2-2,15-1,13-2,11-2,10-1,2-4,3-5
1,2,Toronto Raptors,53-19,26-10,27-9,34-11,19-8,9-5,13-3,12-3,...,13-4,6-4,23-10,4-1,10-3,9-7,12-3,7-3,4-1,7-1
2,3,Los Angeles Lakers,52-19,25-10,27-9,16-9,36-10,5-5,4-3,7-1,...,11-7,7-3,25-11,3-1,14-1,9-5,10-4,9-2,4-1,3-5
3,4,Los Angeles Clippers,49-23,27-9,22-14,17-7,32-16,5-3,4-3,8-1,...,12-5,8-3,32-8,4-2,10-4,10-5,9-4,7-4,4-1,5-3
4,5,Boston Celtics,48-24,26-10,22-14,30-13,18-11,9-6,9-4,12-3,...,10-8,6-5,29-7,3-1,10-4,10-3,9-7,9-3,2-3,5-3
5,6,Denver Nuggets,46-27,26-11,20-16,17-11,29-16,6-4,4-4,7-3,...,8-10,9-5,16-13,3-2,10-2,10-6,11-5,6-4,3-3,3-5
6,7,Indiana Pacers,45-28,25-11,20-17,28-19,17-9,9-7,8-7,11-5,...,13-5,5-7,20-12,1-3,11-4,10-5,9-5,5-7,3-2,6-2
7,8,Houston Rockets,44-28,24-12,20-16,16-9,28-19,6-3,4-3,6-3,...,10-8,4-5,22-14,3-1,10-5,10-5,7-7,9-2,1-4,4-4
8,9,Miami Heat,44-29,29-7,15-22,30-13,14-16,9-6,11-3,10-4,...,9-10,6-4,23-11,4-1,9-4,11-4,8-6,6-7,3-2,3-5
9,10,Oklahoma City Thunder,44-28,23-14,21-14,17-9,27-19,5-3,6-4,6-2,...,11-6,8-6,18-10,1-4,6-7,11-4,12-5,7-3,3-1,4-4


In [11]:
df.to_csv(f"Expanded_Standings/{year}.csv")